In [1]:
import sys
sys.path.insert(0, "/home/allen/Documents/caffe/python")
import caffe
import numpy as np

import caffe.proto.caffe_pb2 as caffepb
from google.protobuf import text_format
import tensorflow as tf

In [ ]:
deploy = "/media/allen/mass/caffe-tensorflow/mobilenet_multitask_20190322.temp.cp.prototxt"
caffemodel = "/media/allen/mass/caffe-tensorflow/mobilenet_multitask_20190322.temp.caffemodel"
# deploy = "/media/allen/mass/caffe-tensorflow/20190220_fmobilenet_fc_align.temp.prototxt"
# caffemodel = "/media/allen/mass/caffe-tensorflow/20190220_fmobilenet_fc_align.temp.caffemodel"
# net = caffe.Net(deploy, caffemodel, caffe.TEST)

# net_params = caffepb.NetParameter()
# f = open(deploy, 'rb')
# net_str = f.read()
# text_format.Merge(net_str, net_params)

In [ ]:
class Network():
    def __init__(self, deploy, caffemodel):
        
        net_params = caffepb.NetParameter()
        f = open(deploy, 'rb')
        net_str = f.read()
        text_format.Merge(net_str, net_params)

        self.weights = caffe.Net(deploy, caffemodel, caffe.TEST).params
        self.caffe_graph = net_params
        
        self.caffe_layer_list = {}
        self.set_caffe_layer_list()
        
        self.sess = tf.InteractiveSession()
        self.graph = self.sess.graph
        self.end_points = {}
        self.var_data_map = {}
        
    def show_graph(self):
        for i in self.graph.as_graph_def().node:
            print("{:45} {:20}".format(i.name, i.op))
        
    def set_caffe_layer_list(self):
        for layer in self.caffe_graph.layer:
            self.caffe_layer_list[layer.name] = layer
        
    def get_layer(self, l_name, level='next', target=None):
        next_layers = []
        if level == 'next':
            tops = self.caffe_layer_list[l_name].top
            for layer in self.caffe_graph.layer:
                for top_name in tops:
                    if top_name in layer.bottom:
                        next_layers.append(layer)
        else:
            bottoms = self.caffe_layer_list[l_name].bottom
            for layer in self.caffe_graph.layer:
                for bottom_name in bottoms:
                    if bottom_name in layer.top:
                        next_layers.append(layer)
                    
        if target is not None:
            for layer in next_layers:
                if layer.type == target:
                    return layer
        return next_layers
    
    def record_tf_variable(self, name, data):
        self.var_data_map[name] = data
#         return tf.constant(value=weight, dtype=tf.float32)        
    
    def add_end_points(self, l, output):
        self.end_points[l.name] = output
               
        
    def add(self, l):     
        if l.type == 'Input':
            n, c, h, w = l.input_param.shape[0].dim
            shape = [n, h, w, c]
            output = tf.placeholder(tf.float32, shape=shape, name=l.name)
            self.add_end_points(l, output)
            
        elif l.type == 'BatchNorm':
            scale_layer = self.get_layer(l.name, target='Scale')            
            w1 = self.weights[l.name]
            raw_mean = w1[0].data
            raw_var = w1[1].data
            w2 = self.weights[scale_layer.name]
            raw_scale = w2[0].data
            raw_offset = w2[1].data
            
            mean = self.record_tf_variable(l.name+'/moving_mean', raw_mean)
            var = self.record_tf_variable(l.name+'/moving_variance', raw_var)
            scale = self.record_tf_variable(l.name+'/gamma', raw_scale)
            offset = self.record_tf_variable(l.name+'/beta', raw_offset)
            
            bottoms = self.get_layer(l.name, level='prev')  
            assert len(bottoms) == 1
            inputs = self.end_points[bottoms[-1].name]
            input_shape = inputs.get_shape()
            if len(input_shape) == 2:                
                inputs = tf.reshape(inputs, shape=[input_shape[0], 1, 1, input_shape[1]], name=l.name+"_expand")          
                output = slim.batch_norm(inputs=inputs, center=True, scale=True, is_training=True, activation_fn=None, fused=True, scope=l.name)
                output = tf.reshape(output, shape=[input_shape[0], input_shape[1]], name=l.name+"_reduce")  
            else:
                output = slim.batch_norm(inputs=inputs, center=True, scale=True, is_training=True, activation_fn=None, fused=True, scope=l.name)
            self.add_end_points(scale_layer, output)
        
        elif l.type == 'Convolution':     
            
            params = l.convolution_param
            c_o = params.num_output
            bias = params.bias_term
            group = params.group
            p_h = params.pad_h
            p_w = params.pad_w
            k_h = params.kernel_h
            k_w = params.kernel_w
            s_h = params.stride_h
            s_w = params.stride_w
            dilation = params.dilation
            
            if s_h == 0 or s_w == 0:
                s_h = s_w = params.stride[0]
            if k_h == 0 or k_w == 0:
                k_h = k_w = params.kernel_size[0]
            if len(params.pad) > 0 and params.pad[0] > 0:
                p_h = p_w = params.pad[0]
                
            bottoms = self.get_layer(l.name, level='prev')
            inputs = self.end_points[bottoms[-1].name]
            input_shape = inputs.get_shape()
            
            assert len(input_shape) == 4
            _, _, _, c_i = input_shape
            
            if p_h > 0:
                inputs = tf.pad(inputs, [[0, 0], [p_h, p_h], [p_w, p_w], [0, 0]], name="{}_pad".format(l.name))
                pad_type = 'VALID'
            else:
                if s_h > 1:
                    pad_type = 'SAME'
                else:
                    pad_type = 'VALID'
                    
            raw_w = self.weights[l.name][0].data
            if group == 1:
                raw_w = raw_w.transpose((2, 3, 1, 0))
                kernel = self.record_tf_variable(l.name+'/weights', raw_w)
                output = slim.convolution2d(inputs=inputs, num_outputs=c_o, kernel_size=k_h, stride=s_h, padding=pad_type, scope=l.name, activation_fn=None)
                if bias:
                    raw_b = self.weights[l.name][1].data                  
                else:
                    raw_b = raw_b = np.zeros(c_o)
                
            biases = self.record_tf_variable(l.name+'/biases', raw_b)
            elif group == c_i and group == c_o:                
                raw_w = raw_w.transpose((2, 3, 0, 1))
                kernel = self.record_tf_variable(l.name+'/weights', raw_w)
                output = tf.nn.depthwise_conv2d(inputs, filter=kernel, strides=[1,s_h, s_w,1], rate=[1,1], padding=pad_type, name=l.name)
            else:
                raw_w = raw_w.transpose((2, 3, 1, 0))
                
                # Split the input into groups and then convolve each of them independently
                input_groups = tf.split(axis=3, num_or_size_splits=group, value=inputs)
                convolve = lambda i : output = slim.convolution2d(inputs=i, num_outputs=c_o, kernel_size=k_h, stride=s_h, padding=pad_type, scope=l.name, activation_fn=None)
                output_groups = []
                for i, inputs in enumerate(input_groups)
                    output_groups.append(slim.convolution2d(inputs=inputs, num_outputs=c_o, kernel_size=k_h, stride=s_h, padding=pad_type, scope=l.name+"_i", activation_fn=None))
                    kernel = self.record_tf_variable('{}_{}/weights'.format(l.name, i), raw_w)
                # Concatenate the groups
                output = tf.compat.v1.concat(values=output_groups, axis=3)
                
            
            
            self.add_end_points(l, output)
        
        elif l.type == 'ReLU' or l.type == 'PReLU':
            bottom = self.get_layer(l.name, level='prev')[-1]
            inputs = self.end_points[bottom.name]
            output = tf.nn.relu(inputs, name=l.name)
            self.add_end_points(l, output)
        
        elif l.type == 'InnerProduct':
            params = l.inner_product_param
            c_o = params.num_output
            bias = params.bias_term
            
            bottom = self.get_layer(l.name, level='prev')[-1]
            inputs = self.end_points[bottom.name]
            input_shape = inputs.get_shape()
            if len(input_shape) == 4:
                n, h, w, c = input_shape 
                c_i = h*w*c
                inputs = tf.reshape(inputs, shape=[1, c_i])
            else:
                n, c_i = input_shape
            
            raw_w = self.weights[l.name][0].data
            raw_w = raw_w.transpose((1,0))
            if bias:
                raw_b = self.weights[l.name][1].data
            else:
                raw_b = np.zeros(c_o)
            weights = self.make_tf_variable(l.name+'_fc_w', raw_w)
            biases = self.make_tf_variable(l.name+'_fc_b', raw_b)
            output = tf.compat.v1.nn.xw_plus_b(x=inputs, weights=weights, biases=biases, name=l.name)
            self.add_end_points(l, output)
            
        elif l.type == 'Scale':
            print("Merge into BN")
            
        elif l.type == 'Eltwise':
            bottoms = self.get_layer(l.name, level='prev')
            output = self.end_points[bottoms[-1].name]
            op = l.eltwise_param.operation
            if op == 1:
                for i, bottom in enumerate(bottoms[1:]):
                    if bottom.type == 'BatchNorm':
                        continue
                    with tf.compat.v1.variable_scope("{}_add_{}".format(l.name, i), values=[output]) as scope:
                        output += self.end_points[bottom.name]
            else:
                print("Unsupported layer {}/{} at {}".format(l.type, op, l.name))
                sys.exit(1)
            self.add_end_points(l, output)
        elif l.type == 'Pooling':
            bottoms = self.get_layer(l.name, level='prev')
            inputs = self.end_points[bottoms[-1].name]
            params = l.pooling_param
            p_h = params.pad_h
            p_w = params.pad_w
            k_size = params.kernel_size
            s = params.stride           
            op = params.pool   
            '''
            enum PoolMethod {
                MAX = 0;
                AVE = 1;
                STOCHASTIC = 2;
              }
            '''
            if p_h > 0:
                inputs = tf.pad(inputs, [[0, 0], [p_h, p_h], [p_w, p_w], [0, 0]], name="{}_pad".format(l.name))
                pad_type = 'VALID'
            else:
                if s > 1:
                    pad_type = 'SAME'
                else:
                    pad_type = 'VALID'
                    
            if op == 0:
                output = tf.nn.max_pool2d(inputs, ksize=[1, k_size, k_size, 1], strides=[1, s, s, 1], padding=pad_type, name=l.name)
            if op == 1:
                output = tf.nn.avg_pool2d(inputs, ksize=[1, k_size, k_size, 1], strides=[1, s, s, 1], padding=pad_type, name=l.name)
                
            self.add_end_points(l, output)
            
        elif l.type == 'Sigmoid':
            bottoms = self.get_layer(l.name, level='prev')
            inputs = self.end_points[bottoms[-1].name]
            input_shape = inputs.get_shape()
            if len(input_shape) == 4 and input_shape[1] == input_shape[2] == 1:
                inputs = tf.reshape(inputs, shape=[input_shape[0], input_shape[3]], name=l.name+"_flatten")
            output = tf.nn.sigmoid(inputs, name=l.name)
            self.add_end_points(l, output)
            
        else:
            print("Unsupported layer {} at {}".format(l.type, l.name))
            sys.exit(1)
            
    def build(self):
        for layer in self.caffe_graph.layer:
            self.add(layer)
    
    def allocate_weights(self):
        for name in self.var_data_map.keys():
            with tf.compat.v1.variable_scope("", reuse=True):
                data = self.var_data_map[name]
                var = tf.compat.v1.get_variable(name)
                self.sess.run(var.assign(data))

In [2]:
slim = tf.contrib.slim

img = tf.placeholder(np.float32, [1,7,7,3])
conv_layer = slim.convolution2d(inputs=img, num_outputs=5, kernel_size=3, stride=1, padding='VALID', scope='test_conv',activation_fn=None)

W0911 11:24:10.737673 140472409773824 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
slim = tf.contrib.slim

img = tf.placeholder(np.float32, [1,7,7,3])
bn_layer = slim.batch_norm(inputs=img, center=True, scale=True, is_training=True, activation_fn=None, fused=True, scope='test_bn')

W0911 11:28:34.402771 140119288514304 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
bn_layer

<tf.Tensor 'test_bn/FusedBatchNorm:0' shape=(1, 7, 7, 3) dtype=float32>

In [5]:
slim = tf.contrib.slim

img = tf.placeholder(np.float32, [1,3])
fc_layer = slim.fully_connected(inputs=img, num_outputs=5, activation_fn=None, scope='test_fc')

In [6]:
graph = tf.get_default_graph()
for i in graph.as_graph_def().node:
    print("{:45} {:20}".format(i.name, i.op))

Placeholder                                   Placeholder         
test_conv/weights/Initializer/random_uniform/shape Const               
test_conv/weights/Initializer/random_uniform/min Const               
test_conv/weights/Initializer/random_uniform/max Const               
test_conv/weights/Initializer/random_uniform/RandomUniform RandomUniform       
test_conv/weights/Initializer/random_uniform/sub Sub                 
test_conv/weights/Initializer/random_uniform/mul Mul                 
test_conv/weights/Initializer/random_uniform  Add                 
test_conv/weights                             VariableV2          
test_conv/weights/Assign                      Assign              
test_conv/weights/read                        Identity            
test_conv/biases/Initializer/zeros            Const               
test_conv/biases                              VariableV2          
test_conv/biases/Assign                       Assign              
test_conv/biases/read           

In [7]:
sess = tf.InteractiveSession()
with tf.compat.v1.variable_scope("", reuse=True):
    data = np.ones((3,3,3,5))
    var = tf.compat.v1.get_variable("test/weights")
    sess.run(var.assign(data))